In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Exploratory Analysis

In [ ]:
df = pd.read_csv('C:\\Users\\alvar\\Documents\\tf_templates\\DATA\\kc_house_data.csv')
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().transpose()

In [ ]:
sns.histplot(data=df,x='price');

In [ ]:
sns.countplot(data=df,x='bedrooms');

In [ ]:
sns.scatterplot(data=df, x='price', y='sqft_living');

In [ ]:
sns.boxplot(data=df, x='bedrooms', y='price');

In [ ]:
len(df) * 0.01

In [ ]:
df_non_top = df.sort_values('price', ascending=False).iloc[216:]
sns.scatterplot(data=df_non_top, x='long', y='lat', hue='price', palette='RdYlGn');

In [ ]:
sns.boxplot(data=df, x='waterfront', y='price');

In [ ]:
df.info()

### Data Engineering

In [ ]:
df = df.drop('id', axis=1)

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [ ]:
sns.boxplot(data=df, x='year', y='price');

In [ ]:
df.groupby('month').mean()['price'].plot();

In [ ]:
df.groupby('year').mean()['price'].plot();

In [ ]:
df = df.drop('date', axis=1)

In [ ]:
df.columns

In [ ]:
df = df.drop('zipcode', axis=1)

### Scaling and Traing-Test Split

In [ ]:
X = df.drop('price', axis=1).values
y = df['price'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
scaler = MinMaxScaler()
# To prevent data leakage from the test set, we only fit our scaler to the training set
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

### Creating a Model and Training

In [ ]:
model = Sequential()

model.add(Dense(units=19, activation='relu'))
model.add(Dense(units=19, activation='relu'))
model.add(Dense(units=19, activation='relu'))
model.add(Dense(units=19, activation='relu'))
model.add(Dense(units=1, activation='linear'))

#### Choosing an optimizer and loss

    # For a multi-class classification problem
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # For a binary classification problem
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # For a mean squared error regression problem
    model.compile(optimizer='rmsprop',
                  loss='mse')

In [ ]:
model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), batch_size=128, epochs=400)

### Evaluation

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot();

In [ ]:
predictions = model.predict(X_test)

In [ ]:
mean_absolute_error(y_test, predictions)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
rmse

In [ ]:
explained_variance_score(y_test, predictions)

In [ ]:
plt.scatter(y_test, predictions);
plt.plot(y_test, y_test, 'r');

In [ ]:
y_test.shape

In [ ]:
errors = y_test.reshape(6480,1) - predictions
sns.histplot(errors);

#### Predicting a new item

In [ ]:
house_num = np.random.randint(100)
single_house = df.drop('price', axis=1).iloc[house_num]

In [ ]:
single_house = scaler.transform(single_house.values.reshape(-1,19))
single_house

In [ ]:
prediction = model.predict(single_house)
prediction

In [ ]:
df.iloc[house_num]